<a href="https://colab.research.google.com/github/ozdemrburak/Nutrition_Assistant_using_Deep_Learning/blob/main/dataset_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("siddhantrout/nutrition5k-dataset")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd

path = "/root/.cache/kagglehub/datasets/siddhantrout/nutrition5k-dataset/versions/1/dishes.xlsx"
df = pd.read_excel(path)
df.head()

In [ ]:
df["dish_id"].is_unique

In [ ]:
df.info()

dishes.xlsx deki data'da total_carb, total_fat, total_protein değerleri bazı görsellerde 0 girilmiş. Ayrıca 3490 resim varken 5006 değer var, sanırım side_angled versiyonunda 5k fotoğraf var ama o açıları kullanmama ihtiyaç yok şimdilik.

In [ ]:
import pickle

images_path = "/root/.cache/kagglehub/datasets/siddhantrout/nutrition5k-dataset/versions/1/dish_images.pkl"
with open(images_path, 'rb') as f:
    images = pickle.load(f)

In [ ]:
images.head()

In [ ]:
len(images["rgb_image"])

In [ ]:
import pandas as pd
df_i = pd.read_excel("/root/.cache/kagglehub/datasets/siddhantrout/nutrition5k-dataset/versions/1/dish_ingredients.xlsx")
df_i.head()

In [ ]:
df_i["dish_id"].is_unique

In [ ]:
df_i.info()

In [ ]:
df_deprecated = df_i.loc[df_i["ingr_name"] == "deprecated"]["dish_id"]

In [ ]:
import pandas as pd

# dish_id'ye göre gruplama ve sayısal kolonları toplama, dishes.xslx'de bazı değerler eksikti. Burada malzemelerin calori vs. değerlerini toplayıp doğru değerleri elde ettik.
df_new = df_i.groupby('dish_id')[['grams', 'calories', 'fat', 'carb', 'protein']].sum().reset_index()

print("Yeni DataFrame:")
print(df_new.head())
print(f"\nYeni DataFrame boyutu: {df_new.shape}")
print(f"\nVeri tipleri:")
print(df_new.dtypes)



In [ ]:
df_new.loc[df_new["calories"] > 1000]

In [ ]:
df_new.info()

In [ ]:
images.loc[images["dish"] == "dish_1558028194"]

In [ ]:
first_image = images["rgb_image"][415]
image_name = images["dish"][415]
print(image_name)

In [ ]:
import matplotlib.pyplot as plt
import io
from PIL import Image
import numpy as np

# Open the byte string as an image using PIL
image = Image.open(io.BytesIO(first_image))

# Convert the image to a NumPy array
image_array = np.array(image)

dish_id = df_new[df_new["dish_id"] == image_name]["dish_id"].values[0]
cal = df_new[df_new["dish_id"] == image_name]["calories"].values[0]
carb = df_new[df_new["dish_id"] == image_name]["carb"].values[0]
fat = df_new[df_new["dish_id"] == image_name]["fat"].values[0]
protein = df_new[df_new["dish_id"] == image_name]["protein"].values[0]

# Display the image
plt.title(f"dish_id: {dish_id} | Calories: {cal}, Carb: {carb}, Fat: {fat}, Protein: {protein}")
plt.imshow(image_array)
plt.axis('off') # Hide axes for a cleaner image display
plt.show()

In [ ]:
images_subset = images[['dish', 'rgb_image']]
df_merged = images_subset.merge(df_new, left_on="dish", right_on="dish_id", how = "inner")

In [ ]:
df_merged.head()

In [ ]:
df_merged.info()

In [ ]:
import matplotlib.pyplot as plt

# Kaç tane resim göstermek istiyorsun? (ör. tüm dataset için len(df_merged))
N = len(df_merged)  # tüm resimler
#N = 10            # sadece ilk 10 için

for i in range(3489, 3490):
    row = df_merged.iloc[i]

    img = row["rgb_image"]
    image = Image.open(io.BytesIO(img))

    cal = row["calories"]
    carb = row["carb"]
    fat = row["fat"]
    protein = row["protein"]
    dish_id = row["dish_id"]

    plt.figure(figsize=(5, 5))
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"id: {dish_id} Calories: {cal}, Carb: {carb}, Fat: {fat}, Protein: {protein}")
    plt.show()


In [ ]:
def remove_dish_ids(df_merged, txt_file_path):
    """
    Set operations kullanarak daha hızlı silme işlemi
    """

    # Txt dosyasını oku
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        dish_ids_to_remove = set(line.strip() for line in file if line.strip())

    print(f"Silinecek dish_id sayısı: {len(dish_ids_to_remove)}")

    # Set operations kullanarak filtreleme
    original_size = len(df_merged)
    mask = ~df_merged['dish_id'].isin(dish_ids_to_remove)
    df_cleaned = df_merged[mask].copy()

    final_size = len(df_cleaned)
    print(f"Orijinal: {original_size} -> Temizlenmiş: {final_size}")
    print(f"Silinen: {original_size - final_size} satır")

    return df_cleaned

In [ ]:
dataset_cleaned = remove_dish_ids(df_merged, "exclusion.txt")

In [ ]:
dataset_cleaned.info()

3490 fotoğraf tek tek incelendiğinde öne çıkanlar şunlar:
1. Bazı besinler kendini çok tekrar ediyor (ör. elma, badem, domates) bunlar veri seti dengesizliğine yol açacağından azaltıldı.
2. 2 adet elma fotoğrafında protein, karbonhidrat değerleri yanlış. Tespit edilen 2 resim silindi.
3. Badem görselleriyle eşleşen karbonhidrat değerleri hatalı. Badem içeren görsellerde kalorinin ve karbonhidrat değerleri biraz fazla tahmin edilecek.
4. Bulanık ve tek görselde çoklu tabak(tabakların biri kısmen gözüküyor ve kısmen gözüken tabaklar hesaba katılmamış) olan görseller temizlendi.
5. Tüm bu gözlemler sonucu 3490 fotoğrafın 57 tanesi temizlendi.